<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=7ab730e1ff742a08b06d92209f315074e503865a9f582d784574fae59af331b4
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-13 10:21:59
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.45 C
-------------------
Today PnL: 20.45 K (0.14%)
Current PnL: -21.21 L (-13.9%)
CY Booked + Current PnL: -7.69 L (-5.04%)
-------------------
Total profit:  1.94 L
Total loss:  -23.15 L
-------------------
Total Booked + Current PnL: 19.29 L (15.36%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.3%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 87.88 L (60.42%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.52%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,102.99,53.0,M-SC,2.73,87482.0,-13295.0,13385.0,0.57,-13.19,15.30,0.09,245.0,-0.99,0.61,14.93,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.00,141.63,83.0,M-SC,3.18,158515.0,-16044.0,16073.0,3.72,-9.19,10.14,0.02,221.0,-1.00,1.10,37.88,OX40N,NTT,PAINTS
2,ABBOTINDIA,35195.00,-10.25,47.0,X-MC,3.35,88725.0,-1953.0,16858.0,0.02,-2.15,19.00,16.44,101.0,-0.12,0.62,17.59,X40,ATH,PHARMA
34,ICICIGI,2252.93,-11.75,62.0,X-MC,7.57,191013.0,16693.0,18509.0,1.25,9.58,9.69,20.19,91.0,0.90,1.33,26.14,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-2.49,65.0,X-LC,8.74,299988.0,34562.0,22229.0,0.30,13.02,7.41,21.40,11.0,1.55,2.08,20.53,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.00,141.63,83.0,M-SC,3.18,158515.0,-16044.0,16073.0,3.72,-9.19,10.14,0.02,221.0,-1.00,1.10,37.88,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-0.25,85.0,X-LC,7.94,251803.0,35.0,53181.0,3.31,0.01,21.12,21.13,27.0,0.00,1.75,35.44,X40,ATH,PAINTS
14,BSOFT,836.99,-7.82,72.0,H-SC,9.02,103708.0,-36945.0,109723.0,2.42,-26.27,105.80,51.74,131.0,-0.34,0.72,19.99,XR,ATH,IT
79,VAIBHAVGBL,521.00,76.31,64.0,H-SC,7.67,152844.0,-29931.0,140999.0,2.30,-16.38,92.25,60.77,125.0,-0.21,1.06,37.65,XR,NTT,JEWELLERY
60,ROUTE,2227.21,-55.56,47.0,H-SC,22.98,73028.0,-60038.0,151920.0,2.16,-45.12,208.03,69.05,118.0,-0.40,0.51,7.60,SR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.0,-54.55,55.0,H-SC,9.55,261538.0,-24967.0,282304.0,-3.72,-8.71,107.94,89.82,134.0,-0.09,1.82,38.37,XY24,NTT,FINANCE
17,CERA,9475.0,-28.16,30.0,H-SC,2.27,130755.0,-45148.0,87174.0,-1.50,-25.67,66.67,23.89,149.0,-0.52,0.91,12.92,OX40N,NTT,CERAMICS
15,CAMPUS,393.0,-25.62,43.0,M-SC,4.12,155024.0,-12751.0,68598.0,-1.14,-7.60,44.25,33.28,210.0,-0.19,1.08,20.15,XY24,NTT,FOOTWEAR
41,IRCTC,1269.0,-35.53,45.0,H-MC,3.10,230841.0,-39667.0,182849.0,-1.08,-14.66,79.21,52.93,83.0,-0.22,1.60,5.68,XY24,BTT,TRAVEL
74,TITAGARH,1548.0,-6.92,47.0,H-SC,4.22,198702.0,-43475.0,154232.0,-0.86,-17.95,77.62,45.74,158.0,-0.28,1.38,29.24,XY24,NTT,ENGINEERING


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.79,54.0,M-LC,5.46,151875.0,930.0,108530.0,-0.11,0.62,71.46,72.51,53.0,0.01,1.06,6.68,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.05,38.0,H-MC,5.89,209375.0,-199.0,74872.0,-0.42,-0.10,35.76,35.63,NaN,-0.00,1.45,16.56,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.0,141.63,83.0,M-SC,3.18,158515.0,-16044.0,16073.0,3.72,-9.19,10.14,0.02,221.0,-1.00,1.10,37.88,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.0,102.99,53.0,M-SC,2.73,87482.0,-13295.0,13385.0,0.57,-13.19,15.30,0.09,245.0,-0.99,0.61,14.93,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.61,50.0,H-SC,2.38,223488.0,-46179.0,82512.0,1.14,-17.12,36.92,13.47,138.0,-0.56,1.55,13.64,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-24.55,48.0,H-MC,6.28,102163.0,-29672.0,71892.0,-0.20,-22.51,70.37,32.03,98.0,-0.41,0.71,14.43,OX40N,NTT,IT
66,SIS,528.0,2023.32,47.0,H-SC,2.14,85680.0,-25352.0,48958.0,-0.27,-22.83,57.14,21.26,156.0,-0.52,0.60,15.76,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.79,54.0,M-LC,5.46,151875.0,930.0,108530.0,-0.11,0.62,71.46,72.51,53.0,0.01,1.06,6.68,XR,NTT,IT
37,INDIAMART,4810.62,-52.59,59.0,H-SC,4.77,132113.0,8777.0,122852.0,-0.47,7.12,92.99,106.72,119.0,0.07,0.92,32.16,AR,ATH,MISC
61,SAMMAANCAP,326.00,-193.18,57.0,M-SC,36.13,161280.0,12060.0,132121.0,1.26,8.08,81.92,96.62,208.0,0.09,1.12,74.54,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.59,59.0,H-SC,4.77,132113.0,8777.0,122852.0,-0.47,7.12,92.99,106.72,119.0,0.07,0.92,32.16,AR,ATH,MISC
84,WIPRO,420.00,-13.79,54.0,M-LC,5.46,151875.0,930.0,108530.0,-0.11,0.62,71.46,72.51,53.0,0.01,1.06,6.68,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.05,38.0,H-MC,5.89,209375.0,-199.0,74872.0,-0.42,-0.10,35.76,35.63,NaN,-0.00,1.45,16.56,AR,ATH,PHARMA
83,WHIRLPOOL,2270.00,-45.46,36.0,M-SC,9.19,113132.0,-1666.0,95710.0,0.23,-1.45,84.60,81.92,223.0,-0.02,0.79,33.93,XR,NTT,DURABLES
47,KPIGREEN,731.05,2.11,42.0,H-SC,10.28,121439.0,-3858.0,62784.0,-0.82,-3.08,51.70,47.03,141.0,-0.06,0.84,52.05,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-17.47,28.0,X-MC,5.77,253605.0,-291.0,82599.0,-0.66,-0.11,32.57,32.42,105.0,-0.00,1.76,14.97,X40,ATH,FMCG
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87158.0,-41512.0,86809.0,-0.70,-32.26,99.60,35.20,219.0,-0.48,0.61,0.31,X40,NTT,FOOTWEAR
1,ABB,7934.00,-43.27,31.0,H-LC,5.15,237120.0,-24499.0,143718.0,0.32,-9.36,60.61,45.57,7.0,-0.17,1.65,0.90,AR,NTT,ELECTRICAL
54,QUESS,986.00,-52.07,31.0,X-SC,37.24,47724.0,-17282.0,168208.0,-0.68,-26.58,352.46,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
53,PGHH,17907.65,-32.93,32.0,X-MC,1.54,194175.0,-6645.0,74447.0,0.57,-3.31,38.34,33.76,80.0,-0.09,1.35,1.72,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.77,44.0,X-LC,0.87,198167.0,-1971.0,22868.0,-0.43,-0.98,11.54,10.44,4.0,-0.09,1.38,4.52,X40,NTT,FMCG
53,PGHH,17907.65,-32.93,32.0,X-MC,1.54,194175.0,-6645.0,74447.0,0.57,-3.31,38.34,33.76,80.0,-0.09,1.35,1.72,X40,ATH,FMCG
65,SIEMENS,4671.50,-3.25,50.0,H-LC,1.62,155500.0,-30595.0,78077.0,1.75,-16.44,50.21,25.51,15.0,-0.39,1.08,14.59,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-8.02,43.0,X-MC,3.23,306873.0,-21118.0,127659.0,0.23,-6.44,41.60,32.48,92.0,-0.17,2.13,4.25,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-6.09,62.0,X-LC,3.28,246201.0,12057.0,36561.0,-0.29,5.15,14.85,20.77,86.0,0.33,1.71,12.42,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.07,31.0,X-SC,37.24,47724.0,-17282.0,168208.0,-0.68,-26.58,352.46,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87158.0,-41512.0,86809.0,-0.70,-32.26,99.60,35.20,219.0,-0.48,0.61,0.31,X40,NTT,FOOTWEAR
1,ABB,7934.00,-43.27,31.0,H-LC,5.15,237120.0,-24499.0,143718.0,0.32,-9.36,60.61,45.57,7.0,-0.17,1.65,0.90,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.93,32.0,X-MC,1.54,194175.0,-6645.0,74447.0,0.57,-3.31,38.34,33.76,80.0,-0.09,1.35,1.72,X40,ATH,FMCG
20,COLPAL,3726.84,-6.96,41.0,X-MC,6.98,216990.0,-46375.0,155690.0,-0.34,-17.61,71.75,41.51,84.0,-0.30,1.51,1.83,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.07,31.0,X-SC,37.24,47724.0,-17282.0,168208.0,-0.68,-26.58,352.46,232.18,198.0,-0.10,0.33,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.36,44.0,X-SC,3.56,81301.0,-63859.0,139789.0,-0.49,-43.99,171.94,52.31,136.0,-0.46,0.56,8.83,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87158.0,-41512.0,86809.0,-0.70,-32.26,99.60,35.20,219.0,-0.48,0.61,0.31,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.25,47.0,X-MC,3.35,88725.0,-1953.0,16858.0,0.02,-2.15,19.00,16.44,101.0,-0.12,0.62,17.59,X40,ATH,PHARMA
33,HONAUT,58357.33,-24.36,37.0,X-SC,0.97,106650.0,-21288.0,68427.0,0.15,-16.64,64.16,36.84,143.0,-0.31,0.74,9.18,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-26.44,61.0,X-LC,8.78,293073.0,-52883.0,119574.0,-0.45,-15.29,40.80,19.28,1.0,-0.44,2.04,8.34,X40,ATH,IT
40,INFY,2275.00,-15.65,64.0,X-LC,4.33,328340.0,15452.0,156224.0,-0.66,4.94,47.58,54.87,3.0,0.10,2.28,13.63,X40,BTT,IT
42,ITC,452.00,-38.77,44.0,X-LC,0.87,198167.0,-1971.0,22868.0,-0.43,-0.98,11.54,10.44,4.0,-0.09,1.38,4.52,X40,NTT,FMCG
81,VBL,671.64,-19.29,45.0,X-LC,5.86,290928.0,-24914.0,137580.0,-0.72,-7.89,47.29,35.67,5.0,-0.18,2.02,4.87,X40N,ATH,FMCG
1,ABB,7934.00,-43.27,31.0,H-LC,5.15,237120.0,-24499.0,143718.0,0.32,-9.36,60.61,45.57,7.0,-0.17,1.65,0.90,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.00,308.96,80.0,L-MC,6.67,50194.0,-27012.0,52408.0,1.87,-34.99,104.41,32.89,258.0,-0.52,0.35,38.23,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,46.0,L-SC,22.26,78029.0,-35520.0,75571.0,0.47,-31.28,96.85,35.27,268.0,-0.47,0.54,90.98,XR,NTT,HOTELS
7,ASIANTILES,137.00,7700.00,71.0,L-SC,15.95,85587.0,-8223.0,84842.0,0.22,-8.77,99.13,81.67,269.0,-0.10,0.59,65.74,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.15,49.0,H-SC,7.51,91710.0,-6270.0,27871.0,-0.75,-6.40,30.39,22.05,152.0,-0.22,0.64,33.31,XR,ATH,FINANCE
37,INDIAMART,4810.62,-52.59,59.0,H-SC,4.77,132113.0,8777.0,122852.0,-0.47,7.12,92.99,106.72,119.0,0.07,0.92,32.16,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,521.00,76.31,64.0,H-SC,7.67,152844.0,-29931.0,140999.0,2.30,-16.38,92.25,60.77,125.0,-0.21,1.06,37.65,XR,NTT,JEWELLERY
38,INDIGOPNTS,1408.00,141.63,83.0,M-SC,3.18,158515.0,-16044.0,16073.0,3.72,-9.19,10.14,0.02,221.0,-1.00,1.10,37.88,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-0.25,85.0,X-LC,7.94,251803.0,35.0,53181.0,3.31,0.01,21.12,21.13,27.0,0.00,1.75,35.44,X40,ATH,PAINTS
7,ASIANTILES,137.00,7700.00,71.0,L-SC,15.95,85587.0,-8223.0,84842.0,0.22,-8.77,99.13,81.67,269.0,-0.10,0.59,65.74,XR,NTT,CERAMICS
14,BSOFT,836.99,-7.82,72.0,H-SC,9.02,103708.0,-36945.0,109723.0,2.42,-26.27,105.80,51.74,131.0,-0.34,0.72,19.99,XR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.79
1,25,44.50
2,50,76.26


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.54
LC    27.97
MC    27.04
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.41
X40      20.82
X40N     12.35
XY25     10.16
XR        9.80
AR        9.14
OX40N     6.59
X200      1.84
SR        1.05
MH        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.92
X-MC    22.93
X-LC    19.96
M-SC    12.57
X-SC     5.59
H-LC     4.57
H-MC     3.76
M-LC     2.45
L-SC     1.46
L-LC     0.99
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.75,-4.36,38.10
IT,12.79,-14.32,75.51
FINANCE,12.06,-9.06,58.50
MISC,7.68,-17.83,77.26
PAINTS,6.00,-6.82,22.75
ELECTRICAL,5.60,-12.09,51.84
INSURANCE,4.51,2.68,31.61
PHARMA,3.48,0.41,33.00
AUTO,3.19,-51.06,113.59


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3219219.0,20
AR,1256424.0,10
XR,1247063.0,13
X40,974373.0,14
X40N,689776.0,9
OX40N,536789.0,9
XY25,471966.0,7
SR,270189.0,2
MH,62784.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3389673.0,24
X-MC,1385628.0,16
M-SC,1236387.0,15
X-LC,780327.0,11
X-SC,545404.0,6
H-MC,329613.0,3
H-LC,280736.0,3
L-SC,249506.0,3
M-LC,154365.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1174248.0      6
           AR         837305.0      5
           XR         782970.0      7
M-SC       XY24       725445.0      6
X-MC       XY24       557609.0      4
           X40        483885.0      8
X-LC       X40        403679.0      5
X-SC       X40N       290387.0      4
H-SC       SR         270189.0      2
           OX40N      262177.0      3
X-LC       X40N       240202.0      3
H-LC       AR         221795.0      2
X-MC       XY25       184947.0      2
H-MC       XY24       182849.0      1
X-SC       XY24       168208.0      1
L-SC       XR         160413.0      2
X-MC       X40N       159187.0      2
M-SC       XR         142742.0      2
           XY25       132121.0      1
           AR         122452.0      2
           OX40N      113627.0      4
M-LC       XR         108530.0      1
L-SC       OX40N       89093.0      1
X-SC       X40         86809.0      1
H-MC       AR          74872.0      1
X-LC       XY25        72420.0      2
H-MC       OX40N       71892.0      1
X-LC       XY24        64026.0      1
H-SC       MH          62784.0      1
H-LC       X200        58941.0      1
L-MC       XR          52408.0      1
M-LC       XY25        45835.0      1
L-LC       XY25        36643.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
